## Introduction

This notebook demonstrates various approaches to financial sentiment analysis using the Financial PhraseBank dataset. We explore multiple machine learning and deep learning techniques, progressing from basic to advanced methods:

1. Traditional ML with embeddings
   - Multi-layer Perceptron (MLP)
   - XGBoost (Gradient Boosting)

2. Sequential Models
   - Gated Recurrent Unit
   - Bidirectional Gated RNNs
   - Transformer Architecture

3. Modern Approaches
   - Pre-trained FinBERT

Each section includes implementation, evaluation metrics, and performance comparison. By the end of this notebook, you'll understand the trade-offs between different approaches to sentiment analysis.

## Setup and Requirements

This notebook requires the following packages:

- transformers (4.30+)
- datasets (2.10+)
- nltk (3.8+)
- xgboost (2.0+)
- sentence_transformers (2.2+)
- keras (3.0+)
- torch (2.0+)

We'll first set up our environment and configure the backend (TensorFlow or PyTorch) based on the operating system.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


from transformers import AutoTokenizer, AutoModel, BertTokenizerFast, BertModel
from sentence_transformers import SentenceTransformer
import xgboost as xgb

import datasets
import keras
import torch
sns.set_theme(style="whitegrid")

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')

In [ ]:
import platform
import os

# Check operating system
is_macos = platform.system() == 'Darwin'

if is_macos:
    # For MacOS, use TensorFlow backend
    os.environ['KERAS_BACKEND'] = 'tensorflow'
    import tensorflow as tf
    print("Using TensorFlow backend on MacOS")
    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
else:
    # For other OS, use PyTorch backend
    os.environ['KERAS_BACKEND'] = 'torch'
    import torch
    print("Using PyTorch backend")
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}")

## Dataset Description

The Financial PhraseBank dataset is a comprehensive collection of financial texts with sentiment annotations:

- **Size**: 4,840 sentences from English financial news articles
- **Labels**: Three-class sentiment classification (positive, negative, neutral)
- **Annotation**: 5-8 annotators per sentence with varying agreement levels
- **Source**: News articles about OMX Helsinki-listed companies
- **Annotators**: Finance experts (researchers and master's students from Aalto University)

### Importance
The dataset addresses the scarcity of high-quality annotated data in financial sentiment analysis, providing a benchmark for model evaluation. The annotations are particularly valuable as they come from individuals with domain expertise in finance.

### Dataset Variants
The dataset offers different versions based on annotator agreement:
- 50% agreement (used in this notebook)
- 66% agreement
- 75% agreement
- 100% agreement


### Data Loading, Inspection, and Preprocessing


In [ ]:
# Load and preprocess the dataset
def load_financial_phrasebank():
    """
    Load and preprocess the Financial PhraseBank dataset from Hugging Face.
    
    Returns:
        pd.DataFrame: Processed dataset with text and sentiment columns
    """
    # Load dataset from Hugging Face
    dataset = datasets.load_dataset('financial_phrasebank', 'sentences_50agree', trust_remote_code=True)
    
    # Convert to pandas DataFrame
    data = pd.DataFrame(dataset['train'])
    
    # Map numerical labels to text labels
    data['sentiment'] = data['label'].map({0: 'negative', 1: 'neutral', 2: 'positive'})
    
    # Rename sentence column to text for consistency
    data = data.rename(columns={'sentence': 'text'})
    
    # Display basic information about the dataset
    print("Dataset Shape:", data.shape)
    print("\nSample entries:")
    display(data.head())
    
    # Show class distribution
    print("\nSentiment Distribution:")
    sentiment_dist = data['sentiment'].value_counts(normalize=True)
    print(sentiment_dist.round(3))
    
    # Create a pie chart of sentiment distribution
    plt.figure(figsize=(10, 6))
    plt.pie(sentiment_dist.values, labels=sentiment_dist.index, autopct='%1.1f%%')
    plt.title('Sentiment Distribution in Financial PhraseBank')
    plt.show()
    
    return data

# Load the dataset
print("Loading Financial PhraseBank dataset from Hugging Face...")
data = load_financial_phrasebank()

### Data Splitting

The dataset doesn't come with a predefined train/validation/test split, so we'll create our own using an 80/10/10 split ratio.

In [ ]:
# Create train/validation/test splits
from sklearn.preprocessing import LabelEncoder

def prepare_data_splits(data, test_size=0.2, val_size=0.25, random_state=42):
    """
    Prepare data splits and encode labels.
    
    Args:
        data (pd.DataFrame): Input dataset
        test_size (float): Proportion of data for testing
        val_size (float): Proportion of training data for validation
        random_state (int): Random seed for reproducibility
    
    Returns:
        tuple: (X_train, X_val, X_test, y_train, y_val, y_test)
    """
    # Encode labels
    le = LabelEncoder()
    y = le.fit_transform(data['sentiment'])
    
    # First split: training + validation vs test
    X_temp, X_test, y_temp, y_test = train_test_split(
        data['text'].values,
        y,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )
    
    # Second split: training vs validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp,
        y_temp,
        test_size=val_size,
        random_state=random_state,
        stratify=y_temp
    )
    
    print("Data split sizes:")
    print(f"Training: {len(X_train)} samples")
    print(f"Validation: {len(X_val)} samples")
    print(f"Test: {len(X_test)} samples")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Create data splits
X_train, X_val, X_test, y_train, y_val, y_test = prepare_data_splits(data)

## Data Modeling

To address the sentiment analysis task using machine learning, we employ modern approaches that leverage **text embeddings** — a cornerstone technique in deep learning–based natural language processing.

In this notebook, we explore three complementary modeling directions:

1. **Sentence-level embeddings**  
   Each sentence is encoded into a fixed-size vector (e.g., 384 dimensions).  
   These embeddings are used as input features for traditional machine learning classifiers such as **MLP** and **XGBoost**.

2. **Token-level embeddings with sequence models**  
   Texts are tokenized into individual tokens, each represented as an embedding vector forming a sequence.  
   Sequence models such as **RNN-based architectures** and **Transformers** are then employed to capture temporal and contextual dependencies within the text.

3. **Pretrained sentiment models (inference only)**  
   We also apply **pretrained sentiment analysis models**, such as **FinBERT**, directly to the test data without additional training.  
   This approach allows us to benchmark performance against state-of-the-art, domain-specific models trained on large financial text corpora.



### Sentence and Word Embeddings Using SentenceTransformers

[**SentenceTransformers**](https://www.sbert.net/) is a powerful library that provides state-of-the-art pretrained models for generating text embeddings.  
In this section, we use it for two complementary purposes:

1. **Sentence-level embeddings**  
   - Convert each full sentence into a fixed-length vector (384 dimensions).  
   - Capture the semantic meaning of complete financial statements.  
   - Serve as input features for traditional machine learning models such as **XGBoost** and **MLP**.

2. **Word-level embeddings**  
   - Generate embeddings for individual tokens within each sentence.  
   - Preserve sequential and contextual information for deep learning models.  
   - Used as input for **GRU**, **LSTM**, and **Transformer** architectures.

#### Model Details
- **Base model:** `all-MiniLM-L6-v2`  
- **Embedding dimension:** 384  
- **Advantages:**  
  - Fast inference time  
  - Strong semantic performance on financial text  
  - Memory efficient and lightweight

These embeddings form the **foundation for all classification models** developed in this notebook.  
Next, we'll load the SentenceTransformer model and ensure it runs on the appropriate compute device (CPU or GPU).


In [ ]:
# Load SentenceTransformer model with appropriate device placement
try:
    from sentence_transformers import SentenceTransformer
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # Handle device placement based on backend
    if is_macos:
        with tf.device(device):
            model = embedding_model
    else:
        model = embedding_model.to(device)
        
    print("Embedding model loaded successfully")
except Exception as e:
    print(f"Error loading embedding model: {e}")

### Traditional Models with Sentence-Level Embeddings

In this section, we implement and evaluate **traditional machine learning models** using the generated sentence embeddings.

#### Generating Sentence Embeddings

In [ ]:
# Generate sentence embeddings for train, validation, and test sets
print("Generating sentence embeddings for all splits...")

# Generate embeddings for each split
X_train_emb = embedding_model.encode(
    X_train,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

X_val_emb = embedding_model.encode(
    X_val,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

X_test_emb = embedding_model.encode(
    X_test,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("\nEmbedding shapes:")
print(f"Training: {X_train_emb.shape}")
print(f"Validation: {X_val_emb.shape}")
print(f"Test: {X_test_emb.shape}")
print(f"\nEach sentence is represented by a {X_train_emb.shape[1]}-dimensional vector")

Let's visualize the sentence embeddings using **t-SNE**, a dimensionality reduction technique for visualization that you'll learn in the **Unsupervised Learning** module.


In [ ]:
# Visualize embeddings using t-SNE
from sklearn.manifold import TSNE

# Combine embeddings and labels for visualization
all_embeddings = np.vstack([X_train_emb, X_val_emb, X_test_emb])
all_labels = np.concatenate([y_train, y_val, y_test])

# Reduce dimensionality to 2D using t-SNE
print("Reducing dimensionality with t-SNE...")
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(all_embeddings)

# Create scatter plot with discrete colors
plt.figure(figsize=(12, 8))

# Define colors and labels for each sentiment
colors = ['red', 'gray', 'green']
sentiments = ['Negative', 'Neutral', 'Positive']

for i, (color, sentiment) in enumerate(zip(colors, sentiments)):
    mask = all_labels == i
    plt.scatter(embeddings_2d[mask, 0], embeddings_2d[mask, 1],
               c=color, label=sentiment, alpha=0.6)

plt.title('t-SNE Visualization of Sentence Embeddings')
plt.xlabel('t-SNE dimension 1')
plt.ylabel('t-SNE dimension 2')
plt.legend(title='Sentiment')
plt.grid(True)
plt.show()


### Task 1: MLP for sentiment classification

Construct a feed-forward neural network that achieves a test accuracy of at least 74.5%

### Task 2: Gradient Boosted Trees for Multiclass Classification

Explore optimized gradient boosting algorithms — **XGBoost**, **LightGBM**, or **CatBoost** — to build a model that achieves performance comparable to the **MLP** model.

### Sequential Models with Word-Level Embeddings

Unlike traditional machine learning approaches that use fixed sentence embeddings, sequential models process text as a **sequence of words**, preserving word order and contextual relationships.  
In this section, we implement three powerful sequential architectures:

1. **Gated RNNs**  
   - A variant of the vanilla RNN  
   - Efficient for processing sequential data  
   - Effective at capturing short-term dependencies  

2. **Bidirectional Gated RNNs**  
   - Process sequences in both forward and backward directions  
   - Capture both past and future context  
   - Well-suited for natural language understanding tasks  

3. **Transformer** *(implemented for you)*  
   - Employs a self-attention mechanism  
   - Enables parallel processing of sequences  
   - Achieves state-of-the-art performance on modern NLP tasks
`


#### Text Preprocessing for Sequential Models

To prepare text data for sequential models, we need to:

1. **Tokenization**
   - Break sentences into individual words/tokens
   - Preserve meaningful linguistic units
   - Handle special characters and punctuation

2. **Sequence Padding**
   - Ensure all sequences have same length
   - Pad shorter sequences with zeros
   - Truncate longer sequences to max length

3. **Word-Level Embeddings**
   - Convert each token to fixed-size vector
   - Capture semantic relationships between words
   - Use pre-trained embeddings for better representation

We'll use NLTK's tokenizer for word segmentation and then create fixed-length sequences suitable for our models.

In [ ]:
import nltk
nltk.download('punkt_tab')  # Download the necessary data for tokenization
from nltk.tokenize import word_tokenize
word_tokenize('hello world!')

Check the sentence length distribution 

In [ ]:
# Analyze sentence length distribution
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate sentence lengths using the tokenized text
sentence_lengths = [len(word_tokenize(text)) for text in X_train]

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Histogram with KDE
sns.histplot(sentence_lengths, bins=30, kde=True, color='skyblue', ax=ax1)
ax1.set_title('Distribution of Sentence Lengths')
ax1.set_xlabel('Number of Words')
ax1.set_ylabel('Count')
ax1.grid(True)

# Plot 2: Box plot
sns.boxplot(y=sentence_lengths, color='skyblue', ax=ax2)
ax2.set_title('Sentence Length Box Plot')
ax2.set_ylabel('Number of Words')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Print statistical summary
print("\nSentence Length Statistics:")
print(f"Minimum length: {min(sentence_lengths)} words")
print(f"Maximum length: {max(sentence_lengths)} words")
print(f"Mean length: {sum(sentence_lengths)/len(sentence_lengths):.1f} words")
print(f"Median length: {sorted(sentence_lengths)[len(sentence_lengths)//2]} words")
print(f"\nPercentiles:")
for p in [10, 25, 75, 90, 95]:
    percentile = np.percentile(sentence_lengths, p)
    print(f"{p}th percentile: {percentile:.1f} words")

# Calculate how many sentences would be affected by different max lengths
for max_len in [30, 40, 50, 60]:
    affected = sum(1 for x in sentence_lengths if x > max_len)
    print(f"\nSentences longer than {max_len} words: {affected} ({affected/len(sentence_lengths):.1%})")

sentence different length, we need to pad them to make them the same length

In [ ]:
# Tokenization and padding function for sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def process_text_sequences(texts, max_length=50, padding='post', truncating='post'):
    """
    Convert texts to padded sequences of tokens.
    
    Args:
        texts: List of text strings
        max_length: Maximum sequence length (default: 50)
        padding: 'pre' or 'post' padding (default: 'post')
        truncating: 'pre' or 'post' truncation (default: 'post')
    
    Returns:
        List of padded token sequences
    """
    # Tokenize all texts
    sequences = [word_tokenize(text) for text in texts]
    
    # Convert tokens to numpy arrays with padding
    padded_sequences = pad_sequences(
        sequences=[s[:max_length] for s in sequences],  # Truncate if needed
        maxlen=max_length,
        padding=padding,
        truncating=truncating,
        dtype=object,  # Use object dtype for string tokens
        value=''  # Use empty string as padding token
    )
    
    return padded_sequences

# Process training, validation, and test sets
print("Processing sequences...")
X_train_padded = process_text_sequences(X_train)
X_val_padded = process_text_sequences(X_val)
X_test_padded = process_text_sequences(X_test)

# Print shapes and example
print(f"\nSequence shapes:")
print(f"Training: {X_train_padded.shape}")
print(f"Validation: {X_val_padded.shape}")
print(f"Test: {X_test_padded.shape}")

# Show an example sequence
print("\nExample padded sequence:")
print(X_train_padded[0])

In [ ]:
from functools import cache
@cache
def encode_words(text):
    return embedding_model.encode(text)

In [ ]:
%%time
# Convert padded sequences to embeddings
def create_embedding_sequences(sequences, max_length=50):
    """
    Convert sequences of words to sequences of embeddings using cached word vectors.
    Ensures all sequences have the same length through padding.
    
    Args:
        sequences: List of word sequences (padded)
        max_length: Maximum sequence length (default: 50)
    Returns:
        numpy array of shape (n_sequences, max_length, embedding_dim)
    """
    n_sequences = len(sequences)
    embedding_dim = 384  # Known dimension from the model
    
    # Initialize the output array with zeros
    embedded_seqs = np.zeros((n_sequences, max_length, embedding_dim))
    
    for i, seq in enumerate(sequences):
        # Get embeddings for non-empty tokens
        valid_tokens = [word for word in seq if word != '']
        # Truncate if necessary
        valid_tokens = valid_tokens[:max_length]
        # Create embeddings for valid tokens
        seq_embeddings = [encode_words(word) for word in valid_tokens]
        
        # Add embeddings to the output array with padding
        for j, embedding in enumerate(seq_embeddings):
            if j < max_length:
                embedded_seqs[i, j] = embedding
    
    return embedded_seqs

# Create embeddings for each split
print("Creating word embeddings for training set...")
X_train_embedded = create_embedding_sequences(X_train_padded)
print("Creating word embeddings for validation set...")
X_val_embedded = create_embedding_sequences(X_val_padded)
print("Creating word embeddings for test set...")
X_test_embedded = create_embedding_sequences(X_test_padded)

print("\nEmbedding shapes:")
print(f"Training: {X_train_embedded.shape}")
print(f"Validation: {X_val_embedded.shape}")
print(f"Test: {X_test_embedded.shape}")

### Task 3: Build a Gated RNN Model (LSTM or GRU) to Achieve Test Accuracy Above 75%

### Task 4: Build a Bidirectional Gated RNN Model to Achieve Test Accuracy Above 76%


At this point, you’ve completed all your assigned tasks.  
To give you a taste of **state-of-the-art models and large language models (LLMs)**, the **Transformer** and **pretrained models** have been implemented for you — you only need to run the provided code.  

🎉 **Congratulations on reaching this stage!**



### Transformer Model for Sentiment Analysis

The Transformer architecture, introduced in the paper *“Attention Is All You Need”* (2017), represents a major breakthrough in deep learning for natural language processing tasks.  
Unlike RNN-based models (LSTM or GRU), Transformers process entire sequences in parallel using self-attention mechanisms.

For this particular problem, however, a Transformer is somewhat **overkill**, as the dataset is relatively small and the task can be solved effectively with simpler models.


In [ ]:
# Import required layers for transformer
from keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Input, GlobalAveragePooling1D, Add, Dense
from keras.models import Model
import numpy as np

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Multi-head self attention with residual connection
    attention_output = MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=head_size, 
        dropout=dropout
    )(inputs, inputs)
    attention_output = Dropout(dropout)(attention_output)
    attention_output = Add()([inputs, attention_output])  # Residual connection
    x1 = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward network with residual connection
    ffn_output = Dense(ff_dim, activation="relu")(x1)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)
    ffn_output = Dropout(dropout)(ffn_output)
    x2 = Add()([x1, ffn_output])  # Residual connection
    return LayerNormalization(epsilon=1e-6)(x2)

# Build the transformer model
def build_transformer_model(
    max_seq_length=50,
    embed_dim=384,
    num_heads=4,  # Reduced from 8 to better match embedding dimension
    ff_dim=512,   # Increased feed-forward dimension
    num_transformer_blocks=2,
    mlp_units=[256, 128],  # Increased MLP units
    dropout=0.1,
    mlp_dropout=0.1,
):
    inputs = Input(shape=(max_seq_length, embed_dim))
    x = inputs

    # Add positional encoding if needed (not required if using pre-trained embeddings)
    # x = positional_encoding(x)

    # Transformer blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, embed_dim // num_heads, num_heads, ff_dim, dropout)

    # Global average pooling
    x = GlobalAveragePooling1D()(x)

    # MLP layers with residual connections
    for dim in mlp_units:
        x_residual = x
        x = Dense(dim, activation="relu")(x)
        x = Dropout(mlp_dropout)(x)
        if x_residual.shape[-1] == dim:  # Only add if dimensions match
            x = Add()([x, x_residual])
        x = LayerNormalization(epsilon=1e-6)(x)

    # Classification layer
    outputs = Dense(3, activation="softmax")(x)

    return Model(inputs, outputs)

# Create and compile the model
print("Building transformer model...")
transformer_model = build_transformer_model(
    max_seq_length=50,
    embed_dim=384,
    num_heads=4,      # Adjusted to work better with embedding dimension
    ff_dim=512,       # Increased feed-forward dimension
    num_transformer_blocks=2,
    mlp_units=[256, 128],  # Increased MLP units
    dropout=0.1,
    mlp_dropout=0.1,
)

In [ ]:
# Compile the model
transformer_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Model summary
transformer_model.summary()

# Add early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model with early stopping
print("\nTraining Transformer model...")
history = transformer_model.fit(
    X_train_embedded,  # Use embedded sequences
    y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_val_embedded, y_val),  # Use embedded validation sequences
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Transformer Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Transformer Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Evaluate on test set
print("\nEvaluating on test set...")
test_loss, test_accuracy = transformer_model.evaluate(X_test_embedded, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

### Pre-trained FinBERT for Financial Sentiment Analysis

FinBERT is a domain-specific BERT model pre-trained on financial text, making it particularly well-suited for financial sentiment analysis. 

#### Model Architecture and Pre-training
- **Base Architecture**: Built on BERT-base (12 layers, 768 hidden units, 12 attention heads)
- **Pre-training Data**: 
  - Financial news articles and corporate reports
  - SEC 10-K/Q filings
  - Earnings call transcripts
  - ~46GB of financial text
- **Pre-training Tasks**:
  1. Masked Language Modeling (MLM)
  2. Next Sentence Prediction (NSP)

#### Domain Adaptation
FinBERT adapts BERT for financial text by:
- Learning financial jargon and terminology
- Understanding context-specific meanings
- Capturing market-specific sentiment nuances
- Recognizing financial entities and relationships

#### Implementation Notes
In our implementation:
- Using ProsusAI's FinBERT version
- Fine-tuned for 3-class sentiment classification
- Labels mapped as: [positive → 2, negative → 0, neutral → 1]
- Batch processing for efficiency

This pre-trained model should provide better performance than our custom models, especially given our relatively small dataset.

In [ ]:
# Load FinBERT model for sequence classification
print("Loading FinBERT model...")
model = AutoModelForSequenceClassification.from_pretrained(
    "ProsusAI/finbert", 
    num_labels=3,
    cache_dir="models"  # Cache the model locally
)

In [ ]:
# Load FinBERT tokenizer
print("Loading FinBERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    "ProsusAI/finbert",
    cache_dir="models"  # Cache the tokenizer locally
)

In [ ]:
# Define function for batch prediction
def predict_sentiment_batch(texts, batch_size=32):
    """
    Predict sentiment for a batch of texts using FinBERT.
    
    Args:
        texts (list): List of texts to analyze
        batch_size (int): Size of batches for processing
    
    Returns:
        list: Predicted sentiment labels (0: negative, 1: neutral, 2: positive)
    """
    predictions = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        
        # Tokenize batch
        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        
        # Get predictions
        with torch.no_grad():
            outputs = model(**inputs)
            batch_predictions = outputs.logits.argmax(dim=1).tolist()
            
            # Convert FinBERT labels to our format
            # FinBERT: [positive, negative, neutral] -> Our format: [negative, neutral, positive]
            batch_predictions = [2 if p == 0 else 0 if p == 1 else 1 for p in batch_predictions]
            predictions.extend(batch_predictions)
            
    return predictions

In [ ]:
# Convert numpy array to list of strings and make predictions
print("Making predictions on test set...")
X_test_list = X_test.tolist()  # Convert numpy array to list
predicted = predict_sentiment_batch(X_test_list)

# Display sample predictions
print("\nSample predictions:")
for i in range(5):
    print(f"\nText: {X_test_list[i]}")
    sentiment_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    print(f"Predicted: {sentiment_map[predicted[i]]}")
    print(f"Actual: {sentiment_map[y_test[i]]}")

In [ ]:
# Comprehensive evaluation of FinBERT model
print("FinBERT Model Evaluation")
print("-----------------------")

# Calculate overall metrics
accuracy = (np.array(predicted) == y_test).mean()
print(f"Test Accuracy: {accuracy:.4f}")

# Create and plot confusion matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, predicted)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
plt.title('FinBERT Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:

# Calculate and display misclassified examples
print("\nExample Misclassifications:")
sentiment_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
misclassified = np.where(np.array(predicted) != y_test)[0]
for idx in misclassified[:5]:  # Show first 5 misclassifications
    print(f"\nText: {X_test[idx]}")
    print(f"Predicted: {sentiment_map[predicted[idx]]}")
    print(f"Actual: {sentiment_map[y_test[idx]]}")